In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
source('0_parameters.sh')

source('0_parameters_chr3_chemokine_pos.sh')

source('plot_functions.R')


In [3]:
clumpp <- 1e-5
HGI_cc <- 'B2'
HGI_sx <- 'eur_leave_ukbb_23andme'

pheno <- 'INI10030610'
clumpp_str <- str_replace(sprintf('%.0e', clumpp), '0', '')


In [4]:
ukb_phe_info %>% fread(select=c('GBE_ID', 'GBE_short_name')) -> ukb_phe_info_df
setNames(ukb_phe_info_df$GBE_short_name, ukb_phe_info_df$GBE_ID) -> ukb_phe_info_named_l


In [5]:
ukb_phe_info_named_l[[pheno]]

[1] "Alkaline phosphatase"

In [6]:
HGI_sumstats_f <- file.path(
    data_d, 'plink_format_UKB_cal',
    UKB_cal_f %>%
    str_replace('@@HGI_case_control@@', HGI_cc) %>%
    str_replace('@@HGI_suffix@@', HGI_sx)
)

clump_f <- file.path(
    data_d, 'UKB_PRS_clump',
    basename(HGI_sumstats_f) %>%
    str_replace('.tsv.gz$', str_replace(sprintf('.clump%.0e.clumped.gz', clumpp), '0', ''))
)

UKB_sumstats_f <- file.path(
    data_d, 'UKB_PRS_PheWAS_follow_up_GWAS',
    sprintf('ukb.%s.glm.gz', pheno)
)


In [7]:
clump_f %>% fread() %>% pull(SNP) -> clumped_vars


In [8]:
ukb_cal_annotation_f %>%
fread(colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM')  -> ukb_cal_annotation_df


In [9]:
UKB_sumstats_f %>% fread(colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') %>% filter(ID %in% clumped_vars) -> UKB_sumstats_df


In [10]:
HGI_sumstats_f %>% fread(colClasses = c('#CHROM'='character')) %>%
rename('CHROM'='#CHROM') %>% filter(ID %in% clumped_vars) -> HGI_sumstats_df


In [11]:
inner_join(
    UKB_sumstats_df %>%
    separate(P, c('P_base', 'P_exp'), sep='e', remove=F, fill='right') %>%
    replace_na(list(P_exp='0')) %>% mutate(log10P = log10(as.numeric(P_base)) + as.numeric(P_exp)) %>%
    select(CHROM, POS, ID, BETA, SE, P, log10P),
    HGI_sumstats_df %>%
    separate(P, c('P_base', 'P_exp'), sep='e', remove=F, fill='right') %>%
    replace_na(list(P_exp='0')) %>% mutate(log10P = log10(as.numeric(P_base)) + as.numeric(P_exp)) %>%
    select(CHROM, POS, ID, BETA, SE, P, log10P),
    by=c('CHROM', 'POS', 'ID'),
    suffix=c('_UKB', '_HGI')
) %>%
left_join(
    ukb_cal_annotation_df %>% select(ID, REF, ALT, SYMBOL),
    by='ID'
) %>% mutate(
    is_in_chr3_chemokine_region = (
        CHROM == '3' & 
        ((chr3_chemokine_pos - remove_half_window) <= POS) &
        (POS <= (chr3_chemokine_pos + remove_half_window))
    )
) -> df


In [12]:
df %>% p_HGIpval_vs_UKBpval() -> p_pval
df %>% p_HGIbeta_vs_UKBbeta() -> p_beta

df %>% filter(log10P_UKB > -500) %>% p_HGIpval_vs_UKBpval() -> p_pval_no_ABO
df %>% filter(log10P_UKB > -500) %>% p_HGIbeta_vs_UKBbeta() -> p_beta_no_ABO


In [13]:
out_f <- file.path(
    repo_fig_d, 'UKB_PRS_PheWAS_follow_up_GWAS', sprintf(
        'HGIrel5_%s_%s.clump%s.%s.png',
        HGI_cc, HGI_sx, clumpp_str, pheno
    )
)
    
if(!dir.exists(dirname(out_f))) dir.create(dirname(out_f))

gridExtra::arrangeGrob(
    p_pval, p_beta, p_pval_no_ABO, p_beta_no_ABO, 
    ncol=2,
    top=grid::textGrob(sprintf(
        "Comparison of GWAS associations (%s, %s, clump p1: %s)\n%s (%s)",
        HGI_cc, HGI_sx, clumpp_str,
        ukb_phe_info_named_l[[pheno]], pheno
    ),gp=grid::gpar(fontsize=20))
) -> g

ggsave(file=out_f, g, width=16, height=16)
ggsave(file=str_replace(out_f, '.png$', '.pdf'), g, width=16, height=16)
        


In [14]:
out_f

[1] "../fig/UKB_PRS_PheWAS_follow_up_GWAS/HGIrel5_B2_eur_leave_ukbb_23andme.clump1e-5.INI10030610.png"